# problema di trasporto

Una società di logistica movimenta container vuoti da M magazzini a P porti

La richiesta dei porti (vedi tabella) può essere soddisfatta prelevando i container vuoti da uno qualsiasi dei magazzini. 

![tab1](./tab1.png)

Ogni magazzino, tuttavia, ha una disponibilità limitata di container (vedi tabella)

![tab2](./tab2.png)

La movimentazione avviene attraverso una flotta di camion, ognuno dei quali può trasportare al massimo 2 container. 

Il costo di ogni viaggio dipende dalla distanza che intercorre tra magazzino e porto ed è descritto dalla seguente tabella:

![tab3](./tab3.png)

Come devono essere organizzate le consegne al fine di minimizzare il costo totale di movimentazione?


## modello di programmazione matematica

![img1](./img1.png)

![img2](./img2.png)

In [4]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np

model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

i = [ 'La Spezia', 'Trieste', 'Ancona', 'Napoli', 'Bari' ]
m = [ 20, 15, 25, 33, 21 ]

j = [ 'Padova', 'Arezzo', 'Roma', 'Teramo', 'Lecce', 'Catanzaro' ]
r = [ 10, 12, 20, 24, 18, 40 ]

c = [[ 8700,  3450, 10650, 21450, 24300],
     [11400, 10200,  4950, 11400, 18300],
     [15150, 15900,  8550,  6600, 13500],
     [19650, 13500,  4650,  7200,  9450], 
     [30300, 25200, 16500,  9150,  2850], 
     [32160, 32910, 22410, 11160,  9990]]

c = np.array(c, dtype=np.float32).T

model.i = Set(initialize=i)
model.j = Set(initialize=j)

c_dict = {}
for i, mi in enumerate(model.i):
    for j, mj in enumerate(model.j):
        c_dict[mi, mj] = c[i][j]

m = {mi: m[i] for i, mi in enumerate(model.i)}
r = {mj: r[j] for j, mj in enumerate(model.j)}

model.m = Param(model.i, initialize=m)
model.r = Param(model.j, initialize=r)

model.c = Param(model.i, model.j, initialize=c_dict)

model.x = Var(model.i, model.j, domain=NonNegativeIntegers, initialize=0)
model.y = Var(model.i, model.j, domain=NonNegativeIntegers, initialize=0)

model.pprint()

obj_expr = sum(sum(model.c[i, j]*model.y[i, j] for i in model.i) for j in model.j)

model.cost = Objective(expr = obj_expr, sense=minimize)

model.constraints = ConstraintList()

for i in model.i:
    model.constraints.add(expr = sum(model.x[i, j] for j in model.j) <= disp_magazzini[i])

for j in model.j:
    model.constraints.add(expr = sum(model.x[i, j] for i in model.i) >= rich_porti[j])

for j in model.j:
    for i in model.i:
        model.constraints.add(expr = 2*model.y[i, j] >= model.x[i, j])

#result = SolverFactory('glpk').solve(model, tee=True).write()

model.pprint()

#results = SolverFactory('glpk').solve(model, tee=True)

print ("The solver returned a status of: "+str(results.solver.status))

#model.pprint()
model.display() 

for v in model.component_data_objects(Var, active=True):
    print(v, value(v)) 

print(value(model.cost))

5 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   30 : {('La Spezia', 'Padova'), ('La Spezia', 'Arezzo'), ('La Spezia', 'Roma'), ('La Spezia', 'Teramo'), ('La Spezia', 'Lecce'), ('La Spezia', 'Catanzaro'), ('Trieste', 'Padova'), ('Trieste', 'Arezzo'), ('Trieste', 'Roma'), ('Trieste', 'Teramo'), ('Trieste', 'Lecce'), ('Trieste', 'Catanzaro'), ('Ancona', 'Padova'), ('Ancona', 'Arezzo'), ('Ancona', 'Roma'), ('Ancona', 'Teramo'), ('Ancona', 'Lecce'), ('Ancona', 'Catanzaro'), ('Napoli', 'Padova'), ('Napoli', 'Arezzo'), ('Napoli', 'Roma'), ('Napoli', 'Teramo'), ('Napoli', 'Lecce'), ('Napoli', 'Catanzaro'), ('Bari', 'Padova'), ('Bari', 'Arezzo'), ('Bari', 'Roma'), ('Bari', 'Teramo'), ('Bari', 'Lecce'), ('Bari', 'Catanzaro')}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'La Spezia', 'Trieste', 'Ancona', 'Napoli'

KeyError: "Index '('Padova', 'La Spezia')' is not valid for indexed component 'c'"